In [2]:
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic') 
import tensorflow as tf
import keras
from numpy import array
import pandas as pd
from google.colab import drive
import matplotlib.pyplot as plt
import numpy as np
import math
import matplotlib as mpl
from IPython.core.display import display, HTML
from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from sklearn.preprocessing import MinMaxScaler, RobustScaler
# from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, LSTM, Dropout
from keras.callbacks import ModelCheckpoint, EarlyStopping
import urllib.request
import json
import datetime
import os
from datetime import datetime

In [11]:
from datetime import datetime
from datetime import timedelta


def date_range(start, end):
    start = datetime.strptime(start, "%Y-%m-%d")
    end = datetime.strptime(end, "%Y-%m-%d")
    dates = [(start + timedelta(days=i)).strftime("%Y%m%d") for i in range((end-start).days+1)]
    return dates
dates = date_range("2021-06-01", "2022-06-01")

date_list=dates
url = 'https://www.nongnet.or.kr/api/whlslDstrQr.do?sdate='+date

response = urllib.request.urlopen(url).read()
response = json.loads(response)

data = pd.DataFrame(response['data'])
data

,PUM_NM,LV_NM,TOT_AMT,SAN_NM,SALEDATE,CMP_NM,DAN_NM,WHSAL_NM,SIZE_NM,COST,POJ_NM,TOT_QTY,QTY,KIND_NM,DANQ
0,감귤,.,144000,제주도 제주시,20201105,울산중앙청과,kg,울산도매시장,.,3000,상자,240.0,48,극조생감귤,5.0
1,감귤,.,144000,제주도 제주시,20201105,울산중앙청과,kg,울산도매시장,.,3000,상자,240.0,48,극조생감귤,5.0
2,감귤,.,66000,제주도 제주시,20201105,울산중앙청과,kg,울산도매시장,.,3000,상자,110.0,22,극조생감귤,5.0
3,감귤,.,120000,제주도 제주시,20201105,울산중앙청과,kg,울산도매시장,.,2500,상자,240.0,48,극조생감귤,5.0
4,감귤,.,55000,제주도 제주시,20201105,울산중앙청과,kg,울산도매시장,.,2500,상자,110.0,22,극조생감귤,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118193,새송이,특,85000,경북 성주군,20201105,강서청과,kg,서울강서도매,.,8500,상자,40.0,10,새송이(일반),4.0
118194,사과,.,4698000,경북 영주시,20201105,강서청과,kg,서울강서도매,.,43500,.,2160.0,108,미안마,20.0
118195,사과,.,3243000,경북 영주시,20201105,강서청과,kg,서울강서도매,2개,47000,.,1380.0,69,미안마,20.0
118196,대파,.,-294000,경기 김포시,20201105,강서청과,kg,서울강서도매,1개(내_뿌리),2100,PE대,-140.0,-140,대파(일반),1.0


In [ ]:
for i in date_list:
  date=i
  url = 'https://www.nongnet.or.kr/api/whlslDstrQr.do?sdate='+date # sdate = 날짜
  response = urllib.request.urlopen(url).read()
  response = json.loads(response)
  temp = pd.DataFrame(response['data'])
  data = pd.concat([data, temp], axis=0)
  print(i)

20210601
20210602
20210603
20210604
20210605
20210606
20210607
20210608
20210609
20210610
20210611
20210612
20210613
20210614
20210615
20210616
20210617
20210618
20210619
20210620
20210621
20210622
20210623
20210624
20210625
20210626
20210627
20210628
20210629
20210630
20210701
20210702
20210703
20210704
20210705
20210706
20210707
20210708
20210709
20210710
20210711
20210712
20210713
20210714
20210715
20210716
20210717
20210718
20210719
20210720
20210721
20210722
20210723
20210724
20210725
20210726
20210727
20210728
20210729
20210730
20210731
20210801
20210802
20210803
20210804
20210805
20210806
20210807
20210808
20210809
20210810
20210811
20210812
20210813
20210814
20210815
20210816
20210817
20210818
20210819
20210820
20210821
20210822
20210823
20210824
20210825
20210826
20210827
20210828
20210829
20210830
20210831
20210901
20210902
20210903
20210904
20210905
20210906
20210907
20210908


In [ ]:
data.to_csv('/content/drive/MyDrive/Colab Notebooks/종합설계과제/AT_TSALET_ALL.csv', encoding='utf-8-sig', index=False)

In [ ]:
os.mkdir('./dict')

In [ ]:
from glob import glob
import multiprocessing
import pickle

# 윈도우즈 사용시 함수를 별도의 .py파일로 저장 후 import하여 사용
def preprocessing(tsalet_file):
    unique_pum = [
        '배추', '무', '양파', '건고추','마늘',
        '대파', '얼갈이배추', '양배추', '깻잎',
        '시금치', '미나리', '당근',
        '파프리카', '새송이', '팽이버섯', '토마토',
    ]
    
    unique_kind = [
        '청상추', '백다다기', '애호박', '캠벨얼리', '샤인마스캇'
    ]
    
    train_dict = {
        'date':[]
    }
    
    for sub in unique_pum:
        train_dict[f'{sub}_거래량(kg)'] = []
        train_dict[f'{sub}_가격(원/kg)'] = []
        
    for sub in unique_kind:
        train_dict[f'{sub}_거래량(kg)'] = []
        train_dict[f'{sub}_가격(원/kg)'] = []
        
    tsalet_sample = pd.read_csv(tsalet_file)
    days = sorted(tsalet_sample['SALEDATE'].unique())
    for day in days:
        train_dict['date'].append(day)
        for sub in unique_pum:
            # 날짜별, 품목별, 거래량이 0 이상인 행만 선택
            c = tsalet_sample[(tsalet_sample['SALEDATE']==day) & (tsalet_sample['PUM_NM']==sub) & (tsalet_sample['TOT_QTY']>0)]
            if c.shape[0] == 0:
                train_dict[f'{sub}_거래량(kg)'].append(0)
                train_dict[f'{sub}_가격(원/kg)'].append(0)
            else:
                tot_amt = c['TOT_AMT'].sum().astype(float)
                tot_qty = c['TOT_QTY'].sum().astype(float)
                mean_price = tot_amt/(tot_qty+1e-20)
                train_dict[f'{sub}_거래량(kg)'].append(tot_qty)
                train_dict[f'{sub}_가격(원/kg)'].append(mean_price)
        
        for sub in unique_kind:
            # 날짜별, 품종별, 거래량이 0 이상인 행만 선택
            c = tsalet_sample[(tsalet_sample['SALEDATE']==day) & (tsalet_sample['KIND_NM']==sub) & (tsalet_sample['TOT_QTY']>0)]
            if c.shape[0] == 0:
                train_dict[f'{sub}_거래량(kg)'].append(0)
                train_dict[f'{sub}_가격(원/kg)'].append(0)
            else:
                tot_amt = c['TOT_AMT'].sum().astype(float)
                tot_qty = c['TOT_QTY'].sum().astype(float)
                mean_price = round(tot_amt/(tot_qty+1e-20))
                tot_qty = round(tot_qty, 1)
                train_dict[f'{sub}_거래량(kg)'].append(tot_qty)
                train_dict[f'{sub}_가격(원/kg)'].append(mean_price)
                
    with open(f'/content/dict/{tsalet_file.split("/")[-1].split(".")[0]}.pkl', 'wb') as f:
        pickle.dump(train_dict, f)

tsalet_files = sorted(glob('/content/drive/MyDrive/Colab Notebooks/종합설계과제/AT_TSALET_ALL/*'))

pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
pool.map(preprocessing, tsalet_files)
pool.close()
pool.join()

dict_files = sorted(glob('/content/dict/*.pkl'))

train_dict_list = []
for dict_file in dict_files:
    with open(dict_file, 'rb') as f:
        train_dict = pickle.load(f)
    train_dict_list.append(train_dict)

train = None

for train_dcit in train_dict_list:
    if train is None:
        train = pd.DataFrame(train_dcit)
    else:
        train = pd.concat([train, pd.DataFrame(train_dcit)])

train['date'] = train.date.astype(str).str.replace('-','')
train['date'] = pd.to_datetime(train.date, format='%Y%m%d')

train

In [ ]:
t = ['월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일']
df1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/종합설계과제/train.csv', encoding='utf-8')
df2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/종합설계과제/test_2020-11-05.csv', encoding='utf-8')
df3 = train
df3['date'] = pd.to_datetime(df3['date'])
df3 = df3[df3['date'].notnull()].reset_index(drop=True)
df1 = pd.concat([df1,df2], axis=0)
df1 = pd.concat([df1, df3], axis=0)
df1['date'] = pd.to_datetime(df1['date'])
df1